
> **University of Pisa - M.Sc. Computer Science, Artificial Intelligence**  
> **Human Language Technologies - a.a. 2021/22**
>
> *September, 2022*
>
>**Authors** 
- Irene Pisani *i.pisani1@studenti.unipi.it* (560104)
- Alice Bergonzini *a.bergonzini1@studenti.unipi.it* (560680)


###### **FINAL PROJECT on KEY POINT ANALYSIS (KPA)**
# **Track 1: Key Point Generation**

***Abstract.*** This work aims to describe simple approaches for solving *Key Point Matching* (KPM) and *Key Point Generation* (KPG) tracks proposed at Argument Mining 2021 in the context of the shared task on *Quantitative Summarization and Key Point Analysis* (KPA). \
The presented methods rely on the fine-tuning of some state-of-the-art pre-trained language models both for KPM and KPG subtasks. \
Regarding the KPM task all the models explored were validated using the Hold-Out validation technique and their results were compared to analyze their effectiveness within the task.  Leveraging DeBERTa pre-trained transformer, our best model yields to competitive performance since it achieved  on the test set a mAP Strict and mAP Relaxed score of, respectively, 0,7035 and 0,8857. \
 For the KPG task, a simple baseline based on abstractive summarization approach was provided; our system takes advantage of the pre-trained Google mT5 transformer to generate several points that are finally properly selected.

## **Settings**

- Define Colab GPU to use
- Download TR, VL e TS set from offial IBM reporsitory
- Install required tools
- Import useful libraries
- Load from check point Google mT5 tokenizer and Google mT5 seq2seq model

In [ ]:
!nvidia-smi

Sat Aug 27 17:38:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! git clone "https://github.com/IBM/KPA_2021_shared_task"

Cloning into 'KPA_2021_shared_task'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 44 (delta 14), reused 26 (delta 4), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [ ]:

!pip install datasets -q
!pip install transformers -q
!pip install sentencepiece -q
!pip install rouge_score -q


     |████████████████████████████████| 365 kB 8.4 MB/s 
     |████████████████████████████████| 120 kB 82.0 MB/s 
     |████████████████████████████████| 115 kB 76.1 MB/s 
     |████████████████████████████████| 212 kB 73.7 MB/s 
     |████████████████████████████████| 127 kB 72.2 MB/s 
     |████████████████████████████████| 4.7 MB 7.7 MB/s 
     |████████████████████████████████| 6.6 MB 56.6 MB/s 
     |████████████████████████████████| 1.3 MB 8.7 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import os

import nltk
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator
from datasets import load_dataset, load_metric

In [ ]:
nltk.download('punkt')
metric = load_metric('rouge')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# define device to use: GPU - cuda
device = 'cuda' 

# load from checkpoint model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained('google/mt5-small')       # import pre-trained MT5 tokenizer 
model = AutoModelForSeq2SeqLM.from_pretrained('google/mt5-small')   # import pre-trained MT5 model

## **Dataset**

- Read TR, VL, TS set
- Prepare datasets for passing them to the model

In [ ]:


def load_kpa_data(gold_data_dir, subset):

    # load arguments set, keypoint set and label set from given directory
    arguments_file = os.path.join(gold_data_dir, f"arguments_{subset}.csv")
    key_points_file = os.path.join(gold_data_dir, f"key_points_{subset}.csv")
    labels_file = os.path.join(gold_data_dir, f"labels_{subset}.csv")

    # read arguments set, keypoint set and label set in csv format as pandas dataframes
    arguments_df = pd.read_csv(arguments_file)
    key_points_df = pd.read_csv(key_points_file)
    labels_file_df = pd.read_csv(labels_file)
    
    # label set will be ignored
    # argument set anf keypoint set will be combined as follow

    # define required list to store <arguments, keypoint> pairs informations
    argument, keypoint, argument_id, keypoint_id, stance, topic = [], [], [],[], [], []

    # for each argument and for each key point under the same topic and stance 
    # ---->  create a pair <arguments, keypoint> with all relative info (such as topic, stance, id, text, etc... )
    for arg, arg_id,topic_arg,stance_arg  in zip(arguments_df['argument'],arguments_df['arg_id'],arguments_df['topic'],arguments_df['stance']):
      for kp,kp_id,topic_kp,stance_kp in zip(key_points_df['key_point'],key_points_df['key_point_id'],key_points_df['topic'],key_points_df['stance']):
        if (topic_arg == topic_kp and stance_arg == stance_kp):
          
          argument.append(arg)
          argument_id.append(arg_id)
          keypoint.append(kp)
          keypoint_id.append(kp_id)
          topic.append(topic_arg)
          stance.append(stance_arg)

    # use all the generated pair to create a final dataset (a dataframe)
    dataset_df = pd.DataFrame({'arg_id':argument_id,
                               'key_point_id':keypoint_id,
                               'argument':argument,
                               'keypoint':keypoint,
                               'topic' : topic,
                               'stance': stance})
    # add a supplemntar column to store the concatenation of argument and topic
    dataset_df["arg_topic"] = dataset_df["argument"] + " " + dataset_df["topic"]

    # return final dataset
    return dataset_df, arguments_df, key_points_df, labels_file_df

dataset_directory = "/content/KPA_2021_shared_task/kpm_data"  # directory for dataset used for training and validation set
testset_directory = "/content/KPA_2021_shared_task/test_data" # directory for dataset used for testing set 

# get dataset for training, evaluation and test
tr_data, _, _, _ = load_kpa_data(gold_data_dir = dataset_directory, subset = "train")
vl_data, _, _, _ = load_kpa_data(gold_data_dir = dataset_directory, subset = "dev")
ts_data, _, _, _ = load_kpa_data(gold_data_dir = testset_directory, subset="test")

# show TR, VL, TS set 
print("\nTRAINING SET: "+str(tr_data.shape))
display(tr_data.head())
print("\nVALIDATION SET: "+str(vl_data.shape))
display(vl_data.head())
print("\nTEST SET: "+str(ts_data.shape))
display(ts_data.head())

# save TR, VL, TS set in csv format
dataset_path = "/content/"
tr_data.to_csv(dataset_path + 'KPAgen_tr.csv')
vl_data.to_csv(dataset_path + 'KPAgen_vl.csv')
ts_data.to_csv(dataset_path + 'KPAgen_ts.csv')



TRAINING SET: (24454, 7)


,arg_id,key_point_id,argument,keypoint,topic,stance,arg_topic
0,arg_0_0,kp_0_0,`people reach their limit when it comes to the...,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1,`people reach their limit when it comes to the...
1,arg_0_0,kp_0_1,`people reach their limit when it comes to the...,Assisted suicide reduces suffering,Assisted suicide should be a criminal offence,-1,`people reach their limit when it comes to the...
2,arg_0_0,kp_0_2,`people reach their limit when it comes to the...,People should have the freedom to choose to en...,Assisted suicide should be a criminal offence,-1,`people reach their limit when it comes to the...
3,arg_0_0,kp_0_3,`people reach their limit when it comes to the...,The terminally ill would benefit from assisted...,Assisted suicide should be a criminal offence,-1,`people reach their limit when it comes to the...
4,arg_0_1,kp_0_0,A patient should be able to decide when they h...,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1,A patient should be able to decide when they h...



VALIDATION SET: (4211, 7)


,arg_id,key_point_id,argument,keypoint,topic,stance,arg_topic
0,arg_4_0,kp_4_0,having a school uniform can reduce bullying as...,Children can still express themselves using ot...,We should abandon the use of school uniform,-1,having a school uniform can reduce bullying as...
1,arg_4_0,kp_4_1,having a school uniform can reduce bullying as...,School uniform reduces bullying,We should abandon the use of school uniform,-1,having a school uniform can reduce bullying as...
2,arg_4_0,kp_4_2,having a school uniform can reduce bullying as...,School uniforms encourage discipline or focus ...,We should abandon the use of school uniform,-1,having a school uniform can reduce bullying as...
3,arg_4_0,kp_4_3,having a school uniform can reduce bullying as...,School uniforms saves costs,We should abandon the use of school uniform,-1,having a school uniform can reduce bullying as...
4,arg_4_0,kp_4_4,having a school uniform can reduce bullying as...,School uniforms create a sense of equality/unity,We should abandon the use of school uniform,-1,having a school uniform can reduce bullying as...



TEST SET: (3923, 7)


,arg_id,key_point_id,argument,keypoint,topic,stance,arg_topic
0,arg_0_0,kp_0_0,Routine child vaccinations isn't mandatory sin...,"Routine child vaccinations, or their side effe...",Routine child vaccinations should be mandatory,-1,Routine child vaccinations isn't mandatory sin...
1,arg_0_0,kp_0_1,Routine child vaccinations isn't mandatory sin...,Mandatory vaccination contradicts basic rights,Routine child vaccinations should be mandatory,-1,Routine child vaccinations isn't mandatory sin...
2,arg_0_0,kp_0_2,Routine child vaccinations isn't mandatory sin...,The parents and not the state should decide,Routine child vaccinations should be mandatory,-1,Routine child vaccinations isn't mandatory sin...
3,arg_0_0,kp_0_3,Routine child vaccinations isn't mandatory sin...,Routine child vaccinations are not necessary t...,Routine child vaccinations should be mandatory,-1,Routine child vaccinations isn't mandatory sin...
4,arg_0_1,kp_0_0,Routine child vaccinations should not be manda...,"Routine child vaccinations, or their side effe...",Routine child vaccinations should be mandatory,-1,Routine child vaccinations should not be manda...


In [ ]:
# get the column names for input/target
dataset_columns = ('argument', 'keypoint')
argument_column = dataset_columns[0]
key_point_column = dataset_columns[1]

# load the dataset previously saved
dataset_path = "/content/"
tr_dataset = load_dataset('csv', data_files = dataset_path + '/KPAgen_tr.csv')["train"]
vl_dataset = load_dataset('csv', data_files = dataset_path + '/KPAgen_vl.csv')["train"]
ts_dataset = load_dataset('csv', data_files = dataset_path + '/KPAgen_ts.csv')["train"]

# create a dictionary to wrap together TR, VL, TS set 
data = {'train': tr_dataset, 'validation': vl_dataset, 'test': ts_dataset}
print(data)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'train': Dataset({
    features: ['Unnamed: 0', 'arg_id', 'key_point_id', 'argument', 'keypoint', 'topic', 'stance', 'arg_topic'],
    num_rows: 24454
}), 'validation': Dataset({
    features: ['Unnamed: 0', 'arg_id', 'key_point_id', 'argument', 'keypoint', 'topic', 'stance', 'arg_topic'],
    num_rows: 4211
}), 'test': Dataset({
    features: ['Unnamed: 0', 'arg_id', 'key_point_id', 'argument', 'keypoint', 'topic', 'stance', 'arg_topic'],
    num_rows: 3923
})}


## **Utilities functions**

- preprocessing: preprocess dataset with mT5 tokenizer
- postprocessing: process data to compute required evaluation metric
- compute metrics: compute rouge score to evaluate generated summaries

In [ ]:
max_input_length = 300
max_target_length = 60
padding = "max_length" 

def preprocess_function(data_set):

    inputs = data_set[argument_column]   # get input column
    targets = data_set[key_point_column]  # get target column
    
    # add useful prefix to input, to tell the model which task has to perform
    #prefix = "summarize: "
    #inputs = [prefix + inp for inp in inputs]

    # execute input tokenization
    model_inputs = tokenizer(inputs, 
                             max_length = max_input_length,
                             padding = padding,
                             truncation = True)

    # execute target tokenizatiion
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, 
                           max_length = max_target_length,
                           padding = padding,
                           truncation =True)
    labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def postprocess_text(preds, labels):
    
    # get predictions and labels and split them in different sentence
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing: ROUGE expects a newline after each sentence
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Extract the median scores
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result



## **MT5 fine-tuning, evaluation, prediction: Hugging Face Trainer API**

- model finetuning on TR set
- model evaluation on VL set
- get model prediction (generated summaries/keypoint) on TS set

In [ ]:
# apply preprocessing procedure on TR, VL e TS set
train_dataset = tr_dataset.map(preprocess_function, batched=True)
eval_dataset = vl_dataset.map(preprocess_function, batched=True)
test_dataset = ts_dataset.map(preprocess_function,batched=True)

# define datacollators objects to use for creating batches
data_collator = default_data_collator

# define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir    = '/content/',
    learning_rate = 1e-5,
    evaluation_strategy = "epoch",
    num_train_epochs    = 3,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 16,
    warmup_steps = 500,
    weight_decay = 0.01,
    predict_with_generate = True
)

# initialize Trainer object
trainer = Seq2SeqTrainer(
    model = model,
    args  = training_args,
    train_dataset = train_dataset,
    eval_dataset  = eval_dataset,
    tokenizer     = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

model.cuda()      # pass model to GPU
checkpoint = ''   # define checkpoint 

# runs fine-tuning and save fine-tuned model
train_result = trainer.train(resume_from_checkpoint = None) 
trainer.save_model()

# use model to predict new summary on test set 
test_results = trainer.predict(
      test_dataset = test_dataset,
      metric_key_prefix = "test",
      max_length = max_target_length,
      num_beams = 6)
print(test_results)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: key_point_id, keypoint, argument, topic, stance, Unnamed: 0, arg_topic, arg_id. If key_point_id, keypoint, argument, topic, stance, Unnamed: 0, arg_topic, arg_id are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 24454
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4587


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,7.091400,4.887713,7.049800,0.652300,6.865900,6.863600,13.276900
2,4.152900,4.109730,7.731900,0.701400,7.662400,7.651800,9.354800
3,3.729000,4.108914,6.629500,0.955600,6.445300,6.443700,11.334800


Saving model checkpoint to /content/checkpoint-500
Configuration saved in /content/checkpoint-500/config.json
Model weights saved in /content/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/checkpoint-500/special_tokens_map.json
Copy vocab file to /content/checkpoint-500/spiece.model
Saving model checkpoint to /content/checkpoint-1000
Configuration saved in /content/checkpoint-1000/config.json
Model weights saved in /content/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/checkpoint-1000/special_tokens_map.json
Copy vocab file to /content/checkpoint-1000/spiece.model
Saving model checkpoint to /content/checkpoint-1500
Configuration saved in /content/checkpoint-1500/config.json
Model weights saved in /content/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /content/checkp

PredictionOutput(predictions=array([[    0, 25750, 19878, ...,     0,     0,     0],
       [    0, 25750, 19878, ...,     0,     0,     0],
       [    0, 25750, 19878, ...,     0,     0,     0],
       ...,
       [    0, 34566, 31176, ...,     0,     0,     0],
       [    0, 34566, 31176, ...,     0,     0,     0],
       [    0, 34566, 31176, ...,     0,     0,     0]]), label_ids=array([[   259, 226116,    265, ...,   -100,   -100,   -100],
       [ 49175,  59997,    259, ...,   -100,   -100,   -100],
       [   486,  22552,    305, ...,   -100,   -100,   -100],
       ...,
       [   486,   2279,   1070, ...,   -100,   -100,   -100],
       [   486,   2279,   1070, ...,   -100,   -100,   -100],
       [   486,   2279,   1070, ...,   -100,   -100,   -100]]), metrics={'test_loss': 3.662539005279541, 'test_rouge1': 12.8791, 'test_rouge2': 3.6119, 'test_rougeL': 12.596, 'test_rougeLsum': 12.592, 'test_gen_len': 25.1323, 'test_runtime': 470.9206, 'test_samples_per_second': 8.33, 'tes

In [ ]:
if trainer.is_world_process_zero():
  if training_args.predict_with_generate:
    
    # decode model's predictions
    test_preds = tokenizer.batch_decode(
        test_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
    # split model's predictions
    test_preds = [pred.strip() for pred in test_preds]
    
    # save model's predictions in a txt file
    preds_file = os.path.join('/content/', "test_set_generations.txt")
    with open(preds_file, "w") as file_out:
          file_out.write("\n".join(test_preds))

## **Keypoint Ranking**
- Read all the generated keypoints 
- Assign to each keypoint a rouge-score with its corresponding <argument+topic>
- Rank keypoint based on their score
- Select top 5 keypoint for each topic and stance

In [ ]:
# open file in which predictions are stored
f = open('/content/test_set_generations.txt','r')
preds = f.read().splitlines()

# open test set as dataframe and add a column with all the given predictions
df_test = pd.read_csv(dataset_path + 'KPAgen_ts.csv')
df_test["generated_keypoint"] = preds

# store in a file this dataframe with all the generated keypoint and save it
final_kps = df_test
final_kps.to_csv("/content/final.csv")

In [ ]:
from rouge_score import rouge_scorer

# define a scorer object to assignn rouge 1 score
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

def compute_rouge1_metrics(kp,arg_topic):

  # compute rouge-1 score between keypoint and its correspondinng arguments concatened with the topic
  scores = []
  for i in range(len(kp)):
    score = scorer.score(kp[i], arg_topic[i])
    score = float(score["rouge1"][1])
    scores.append(score)

  return scores

def get_best_score(kps, topic, stance):

  final_kps = []
  final_score = []
  final_topic = []
  final_stance = []
  
  # get list of unique keypoint
  gen_kps_unique = kps["generated_keypoint"].unique()
  
  # for each unique keypoint
  for gen_kp in gen_kps_unique:
    
    # consider all keypoint with same text
    key_point = kps[(kps["generated_keypoint"]==gen_kp)]
    # assign to this keypoint the mean of their score
    best_score = key_point["rouge_score"].max()
    # return keypoint info: text, score, topic, stance
    final_kps.append(gen_kp)
    final_score.append(best_score)
    final_topic.append(topic)
    final_stance.append(stance)
  
  # create a keypoints dataframe
  keypoint_dataframe = pd.DataFrame({"generated_keypoint":final_kps, "score":final_score,"topic":final_topic, "stance":final_stance})
  # order keypoints based on their score
  keypoint_dataframe = keypoint_dataframe.sort_values(by='score', ascending=False)
  # save only the top 5 key point 
  keypoint_dataframe = keypoint_dataframe.iloc[0:5, :]

  return keypoint_dataframe

def keypoint_ranking(kps, test_set):
  
  # initialize an empty dataframe for final keypoints
  final_kps = pd.DataFrame({
      "generated_keypoint": [np.nan],
      "score": [np.nan],
      "topic": [np.nan],
      "stance": [np.nan]
      },
      index=[0]
      )

  # get list of all topic inside TS set
  all_topic = test_set["topic"].unique()

  # for each topic
  for topic in all_topic:
    
    # create a temporaneous test set with only row concerning that topic
    temp_test_set = test_set[(test_set['topic']==topic)]
    # create a temporaneous test set with only row concerning that topic and having positive stance 
    pos_test_set = temp_test_set[(temp_test_set["stance"]==1)]
    # create a temporaneous test set with only row concerning that topic and having negative stance 
    neg_test_set = temp_test_set[(temp_test_set["stance"]==-1)]
   

    # given a temporaneus test set
    # ---> compute rouge score between keypoint (from temp. kp set) and corresponding argument+topic (from temp. test set)
    # repeat the procedure both for positive and negative set  
    pos_test_set["rouge_score"] = compute_rouge1_metrics(pos_test_set["generated_keypoint"].tolist(), pos_test_set["argument"].tolist())
    neg_test_set["rouge_score"] = compute_rouge1_metrics(neg_test_set["generated_keypoint"].tolist(), neg_test_set["argument"].tolist())
    

    # both for postive stance and negative stance set of keypoint keep only top 5 keypoints
    pos_kps_5 = get_best_score(pos_test_set, topic, 1)
    neg_kps_5 = get_best_score(neg_test_set, topic, -1)
    
    # concatanate pos and neg keypoint to obtain top 10 keypoint under this topic
    final_kps = pd.concat([final_kps, pos_kps_5])
    final_kps = pd.concat([final_kps, neg_kps_5])

  # return final keypoint set   
  final_kps = final_kps.iloc[1: , :]
  final_kps = final_kps[["generated_keypoint", "topic", "stance"]]
  final_kps["stance"] = final_kps["stance"].astype(int)
  final_kps = final_kps.reset_index(drop=True)

  display(final_kps)
                                
  return final_kps


final_key_points = keypoint_ranking( final_kps, df_test)

,generated_keypoint,topic,stance
0,The childhood vaccine is essential for the child,Routine child vaccinations should be mandatory,1
1,Private child vaccination is essential to prev...,Routine child vaccinations should be mandatory,1
2,Legalizing vaccination is essential,Routine child vaccinations should be mandatory,1
3,Legalizing vaccine prevents diseases,Routine child vaccinations should be mandatory,1
4,Legalizing vaccination is important,Routine child vaccinations should be mandatory,1
5,Legalizing child vaccinations are expensive,Routine child vaccinations should be mandatory,-1
6,Women should be able to choose to choose their...,Routine child vaccinations should be mandatory,-1
7,Legalizing vaccines are harmful,Routine child vaccinations should be mandatory,-1
8,Women should be able to be able to adopt their...,Routine child vaccinations should be mandatory,-1
9,Legalizing child vaccination,Routine child vaccinations should be mandatory,-1


## Final Keypoints

> Final keypoints are saved in file `generated_keypoints.csv`

In [ ]:
final_key_points.to_csv('/content/generated_keypoints.csv')
print("Final keypoint has been generated and saved in file: generated_keypoints.csv")

Final keypoint has been generated and saved in file: generated_keypoints.csv
